In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
from queue import Queue
from functools import wraps
from operator import add


class Async:

    def __init__(self, func, args):
        self.func = func
        self.args = args


def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)


def inlined_async(func):

    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break

    return wrapper


@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)

    r = yield Async(add, ('hello', 'world'))
    print(r)

    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)

    print('bye')


from multiprocessing import Pool

pool = Pool()
apply_async = pool.apply_async

test()

51:1: E402 module level import not at top of file
5
helloworld
0
2
4
6
8
10
12
14
16
18
bye
